### 1. Formulate the hypothesis

Kiara sets her hypotheses:

Ho:  p = (better than) Po

Ha: p (not better than) Po


Where:

Ho: The new home page design performs better than the old design

Ha: The new home page design does not perform better than the old design

p: The conversion rate of the new design

Po: The conversion rate of the old design (i.e. 13%)

### 2. Conduct power analysis

In [1]:
# [1] Import the ‘statsmodel’ for statistical calculations and
# [1a] the ‘TTestIndPower’ class to calculate the parameters for the analysis:
import statsmodels.stats.api as sms
from statsmodels.stats.power import TTestIndPower


# [2] Specify the three required parameters for the power analysis:
# [2a] Specify ‘alpha’
alpha = 0.05 
# [2b] Specify ‘power’
power = 0.8 
# [2c] Specify ‘effect’ and calculate the minimum effect.
effect = sms.proportion_effectsize(0.13, 0.15) 


# [3] Perform power analysis by using the solve_power() function:
# [3a] Specify an instance of ‘TTestIndPower’
analysis = TTestIndPower() 

# [3b] Calculate the sample size and list the parameters
result = analysis.solve_power(effect, power=power, nobs1=None,
                              ratio=1.0, alpha=alpha) 

# [4] Print the output up to 3 decimal places (with lead-in text)
#print('Sample Size: %.3f' % result)

### 3. Import packages and Load data

In [2]:
# Install the relevant modules:
!pip install scipy

# Import necessary libraries, packages and classes
import pandas as pd
# Import Python's built-in maths module
import math
import numpy as np
# Import statsmodels stats test and tools
import statsmodels.stats.api as sms
# Import the scipy.stats for more stats functions
import scipy.stats as st
# Import Matplotlib for visualisation tools
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
# Read the 'ab.data.csv' file
df = pd.read_csv('ab_data.csv')

# View the DataFrame
df.head()

user_id                   timestamp      group landing_page  converted
0   851104  2017-01-21 22:11:48.556739    control     old_page          0
1   804228  2017-01-12 08:01:45.159739    control     old_page          0
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0
4   864975  2017-01-21 01:52:26.210827    control     old_page          1

In [4]:
# Check the metadata for information about the file
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


### 4. Clean the data

In [5]:
# Use Pandas's duplicated() function to check the user_id column 
print(df[df.user_id.duplicated()])

        user_id                   timestamp      group landing_page  converted
2656     698120  2017-01-15 17:13:42.602796    control     old_page          0
2893     773192  2017-01-14 02:55:59.590927  treatment     new_page          0
7500     899953  2017-01-07 03:06:54.068237    control     new_page          0
8036     790934  2017-01-19 08:32:20.329057  treatment     new_page          0
10218    633793  2017-01-17 00:16:00.746561  treatment     old_page          0
...         ...                         ...        ...          ...        ...
294308   905197  2017-01-03 06:56:47.488231  treatment     new_page          0
294309   787083  2017-01-17 00:15:20.950723    control     old_page          0
294328   641570  2017-01-09 21:59:27.695711    control     old_page          0
294331   689637  2017-01-13 11:34:28.339532    control     new_page          0
294355   744456  2017-01-13 09:32:07.106794  treatment     new_page          0

[3894 rows x 5 columns]


In [6]:
# Use ‘drop_duplicates’ to return the Series without the duplicate values
dropped = df.drop_duplicates(subset = "user_id") 

dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290584 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       290584 non-null  int64 
 1   timestamp     290584 non-null  object
 2   group         290584 non-null  object
 3   landing_page  290584 non-null  object
 4   converted     290584 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 13.3+ MB


In [7]:
# Use ‘dropped.drop’ to remove irrelevant columns from the DataFrame 
# and specify the column names:
# Specify that ‘user_id’ and ‘timestamp’ are columns (i.e. ‘axis 1’) 
final_tab = dropped.drop(["user_id", "timestamp"], axis=1)  

final_tab.head()

group landing_page  converted
0    control     old_page          0
1    control     old_page          0
2  treatment     new_page          0
3  treatment     new_page          0
4    control     old_page          1

In [8]:
# Use ‘crosstab’ to compute a simple cross-tabulation between two variables
pd.crosstab(final_tab["group"], final_tab["landing_page"])

landing_page  new_page  old_page
group                           
control           1006    144226
treatment       144314      1038

In [9]:
# [1] Specify groups to be dropped
final_tab_cleaned = final_tab[((final_tab.group == "control") & (final_tab.landing_page == "old_page")) | (
    (final_tab.group == "treatment") & (final_tab.landing_page == "new_page"))]

# [2] Print the shape of the new ‘final’ table
print(final_tab_cleaned.shape)
final_tab_cleaned["group"].value_counts()

(288540, 3)


treatment    144314
control      144226
Name: group, dtype: int64

In [10]:
# Re-check/compute another simple cross-tabulation
pd.crosstab(final_tab_cleaned["group"], final_tab_cleaned["landing_page"])

landing_page  new_page  old_page
group                           
control              0    144226
treatment       144314         0

### 5. Perform random sampling with Pandas

In [11]:
# Obtain a simple random sample for control and treatment groups with n = 4721; 
# set random_stategenerator seed at an arbitrary value of 22.

# Obtain a simple random sample for the control group.
control_sample = final_tab_cleaned[final_tab_cleaned['group'] == 'control'].sample(n=4721,
                                                                   random_state=22) 
# Obtain a simple random sample for the treatment group.
treatment_sample = final_tab_cleaned[final_tab_cleaned['group'] == 'treatment'].sample(n=4721,
                                                                       random_state=22)

In [12]:
# [1] Join the two samples.  
ab_test = pd.concat([control_sample, treatment_sample], axis=0)  

# [2] Reset the A/B index.
ab_test.reset_index(drop=True, inplace=True) 

# [3] Print the sample table.
ab_test  

group landing_page  converted
0       control     old_page          0
1       control     old_page          0
2       control     old_page          0
3       control     old_page          0
4       control     old_page          0
...         ...          ...        ...
9437  treatment     new_page          0
9438  treatment     new_page          0
9439  treatment     new_page          0
9440  treatment     new_page          0
9441  treatment     new_page          0

[9442 rows x 3 columns]

### 6. Analyse the data

#### 6.a Calculate basic statistics

In [13]:
# [1] import library
from scipy.stats import sem

# [2] Group the ab_test data set by group and aggregate by converted:
conversion_rates = ab_test.groupby('group')['converted']


# [3] Calculate conversion rates by calculating the means of columns STD_p and SE_p:
conversion_rates = conversion_rates.agg([np.mean, np.std,sem])

# [4] Assign names to the three columns.
conversion_rates.columns = ['conversion_rate', 'std_deviation','std_error']  

 # [5] Round the output to 3 decimal places
conversion_rates.style.format('{:.3f}') 

#### 6.b Calculate statistical significance

In [14]:
# [1] Import proportions_ztest and proportion_confint from statsmodels:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

# [2] Create a subset of control and treatment results:
control_results = ab_test[ab_test['group'] == 'control']['converted']
treatment_results = ab_test[ab_test['group'] == 'treatment']['converted']

# [3] Determine the count of the control_results and 
# treatment_result sub-datasets and store them in their respective variables:
n_con = control_results.count()
n_treat = treatment_results.count()

# [4a] Create a variable ‘success’ with the sum of the two data sets
# in a list format: 
successes = [control_results.sum(), treatment_results.sum()]

# [4b] Create a variable ‘nobs’ which stores the values of 
# variables n_con and n_treat in list format: 
nobs = [n_con, n_treat] 

# [5] Use the imported libraries to calculate the statistical values: 
z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes,
                                                                        nobs=nobs,
                                                                        alpha=0.05)

# [6a-d] Print the outputs (with lead-in text):
print(f'Z test stat: {z_stat:.2f}')
print(f'P-value: {pval:.3f}')
print(f'Confidence Interval of 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'Confidence Interval of 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

Z test stat: 1.40
P-value: 0.163
Confidence Interval of 95% for control group: [0.115, 0.134]
Confidence Interval of 95% for treatment group: [0.106, 0.124]


### 7. Summarise and communicate the findings

Recall that Kiara’s  value is 0.05 (5%). Looking at the output, her p-value, 0.163 (16.3%), is much higher than , meaning that Kiara cannot reject her . Why? Recall that -values summarise statistical differences, essentially calculating the probability that the data could deviate from . A -value of 0.163 suggests that the difference between the sample and assumed population means is likely, which in turn suggests that the sample is consistent with the . 

This means that the new design did not perform better than the old design. The confidence interval for the treatment group does not reach 13%, never mind the desired 15%, confirming that the new website design was altogether ineffective in increasing the conversion rate.

Kiara tells the e-commerce company that perhaps it’s time for a rethink!